In [1]:
# -*- coding: utf-8 -*-
# %matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
np.set_printoptions(suppress=True)

from shutil import copyfile
import random
from importlib import reload


from keras.utils import plot_model

from game import Game, GameState
from agent import Agent
from memory import Memory
from model import Residual_CNN
from funcs import playMatches, playMatchesBetweenVersions

import loggers as lg

from settings import run_folder, run_archive_folder
import initialise
import pickle
import config

env = Game()

Using TensorFlow backend.


In [2]:
######## LOAD MODEL ########

# 빈 Neural Network 생성
current_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, (2,) + env.grid_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)
best_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, (2,) +  env.grid_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)

# Model 불러옴
if initialise.INITIAL_MODEL_VERSION != None:
    best_player_version  = initialise.INITIAL_MODEL_VERSION
    print('LOADING MODEL VERSION ' + str(initialise.INITIAL_MODEL_VERSION) + '...')
    m_tmp = best_NN.read(env.name, initialise.INITIAL_MODEL_VERSION, best_player_version)
    current_NN.model.set_weights(m_tmp.get_weights())
    best_NN.model.set_weights(m_tmp.get_weights())
#otherwise just ensure the weights on the two players are the same
else:
    best_player_version = 0
    best_NN.model.set_weights(current_NN.model.get_weights())
    print('NEW MODEL ...')

#copy the config file to the run folder
copyfile('./config.py', run_folder + 'config.py')
plot_model(current_NN.model, to_file=run_folder + 'models/model.png', show_shapes = True)

print('\n')

######## CREATE THE PLAYERS ########

current_player = Agent('current_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, current_NN)
best_player = Agent('best_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, best_NN)
iteration = 0

while 1:
    ######## LOAD MEMORIES ########
    if initialise.INITIAL_MEMORY_VERSION != None:
        memory = pickle.load( open( "./run/memory/memory" + '_' + + str(initialise.INITIAL_MEMORY_VERSION)[0:4] + ".p",   "rb" ) )
        print('USING MEMORY : ' + str(initialise.INITIAL_MEMORY_VERSION))
    else:
        memory = pickle.load( open( "./run/memory/memory_temp" + ".p",   "rb" ) )
        print('USING TEMP MEMORY')
        print('MEMORY SIZE: ' + str(len(memory.ltmemory)))
    
    iteration += 1
    reload(lg)
    reload(config)
    
    print('ITERATION NUMBER ' + str(iteration))
    
    lg.logger_main.info('BEST PLAYER VERSION: %d', best_player_version)
    print('BEST PLAYER VERSION ' + str(best_player_version))

    ######## RETRAINING ########
    print('RETRAINING...')
    current_player.replay(memory.ltmemory)
    print('')
    current_NN.model.save(run_folder + 'models/training_model.h5')

    







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

NEW MODEL ...


USING TEMP MEMORY
MEMORY SIZE: 25354
ITERATION NUMBER 1
BEST PLAYER VERSION 0
RETRAINING...
Epoch 1/1
256/256 [==============================] - 17s 66ms/step - loss: 3.1090 - value_head_loss: 1.2034 - policy_head_loss: 4.8121
Epoch 1/1
160/256 [=================>............] - ETA: 3s - loss: 2.9535 - value_head_loss: 1.0054 - policy_head_loss: 4.6865

KeyboardInterrupt: 